## 6. Advanced Visualizations: Shot Maps

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # Optional, will only affect the color of bars and the grid

from ipywidgets import widgets, interactive

import os

from skimage import io
from skimage.color import rgb2gray
import matplotlib as mpl
from statistics import mean
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/MyDrive/IFT6758/ift6758-project-template-main"
# os.chdir(path)

In [5]:
# Read and concatenate all files
r2016 = pd.read_csv('2016R.csv')
r2017 = pd.read_csv('2017R.csv')
r2018 = pd.read_csv('2018R.csv')
r2019 = pd.read_csv('2019R.csv')
r2020 = pd.read_csv('2020R.csv')

p2016 = pd.read_csv('2016P.csv')
p2017 = pd.read_csv('2017P.csv')
p2018 = pd.read_csv('2018P.csv')
p2019 = pd.read_csv('2019P.csv')
p2020 = pd.read_csv('2020P.csv')

rdata = pd.concat([r2016,r2017,r2018,r2019,r2020])
pdata = pd.concat([p2016,p2017,p2018,p2019,p2020])

rdata['date time']= pd.to_datetime(rdata['date time'])
rdata['coordinates x'] = rdata['coordinates x'].fillna(0)
rdata['coordinates y'] = rdata['coordinates y'].fillna(0)

pdata['date time']= pd.to_datetime(pdata['date time'])
pdata['coordinates x'] = pdata['coordinates x'].fillna(0)
pdata['coordinates y'] = pdata['coordinates y'].fillna(0)

rdata = rdata.drop(['Unnamed: 0'], axis = 1)
pdata = pdata.drop(['Unnamed: 0'], axis = 1)

data = pd.concat([rdata,pdata])

In [41]:
# create Dropdown
season = widgets.Dropdown(
    options=list(data['season'].unique()),
    #value='Select a season',
    description='Season:',
)
game_type = widgets.Dropdown(
    options=['Regular','Playoff'],
    value='Regular',
    description='Game Type:',
)
team = widgets.Dropdown(
    options=list(data['team name'].unique()),
    #value='Select a team',
    description='Team:',
)

# Get league Average
def league_avg(df):
    teams = df['team name'].unique()
    team_shots_per_hr = {}
    for team in teams:
        datas = df.loc[df['team name'] == team]
        gameid_unq = datas['game ID'].unique()
        shots_per_hr = []
        for gameid in gameid_unq:
            data_1 = datas.loc[datas['game ID'] == gameid]
            max_time = data_1['date time'].max()
            min_time = data_1['date time'].min()
            number_shots = len(data_1['coordinates x']) # 38 shots
            play_time = max_time - min_time
            play_time_mins = play_time/ np.timedelta64(1, 'm') # 147 min
            shots_per_hr.append((number_shots/play_time_mins)*60) # (38/147)*60 = 15.5 for one game 
        team_shots_per_hr[team] = mean(shots_per_hr)

    league_avg_shots_phr = mean(list(team_shots_per_hr.values()))
    return league_avg_shots_phr

# Scale the given xy cordinates based on the figure shape
def scale_cordinates(r_max, r_min, t_max, t_min, cordinates):
    new_list = []
    for value in cordinates:
        new_list.append((((value - r_min)/(r_max - r_min))*(t_max - t_min)) + t_min)
    return np.array(new_list)

# Preprocessing and Image creation
def fig_preprocess(data, league_avg_shots_phr, season, game_type, team):
    unique_period = data['period num'].unique()

    # Get unique period list
    for period in unique_period[1::2]:
        for index, row in data.iterrows():
            if row['period num'] == period:
                data.at[index,'coordinates x'] = row['coordinates x'] * -1
                data.at[index,'coordinates y'] = row['coordinates y'] * -1
    # Initialize league data for a team
    league_data = {}
    league_data = {}
    league_data['Shot'] = {}
    league_data['Shot']['x'] = []
    league_data['Shot']['y'] = []
    league_data['Goal'] = {}
    league_data['Goal']['x'] = []
    league_data['Goal']['y'] = []
    
    for index, row in data.iterrows():
        league_data[row["shot or goal"]]['x'].append(row["coordinates x"])
        league_data[row["shot or goal"]]['y'].append(row["coordinates y"])
    
    # Concatenate all Shots and Goals
    league_x_all_shots = league_data['Shot']['x'] + league_data['Goal']['x']
    league_y_all_shots = league_data['Shot']['y'] + league_data['Goal']['y']

    x_max = 100
    x_min = -100
    y_max = 42.5
    y_min = -42.5
    t_min_x = 0
    t_max_x= 1100
    t_min_y = 0
    t_max_y = 467

    # Scale the given xy cordinates based on the figure shape 
    new_x = scale_cordinates(x_max, x_min, t_max_x, t_min_x, league_x_all_shots)
    new_y = scale_cordinates(y_max, y_min, t_max_y, t_min_y, league_y_all_shots)

    #Binning the x and y coordinates
    x = new_x
    y = new_y
    binx = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100]
    biny = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 467]
    ret = stats.binned_statistic_2d(x, y, None, 'count', bins=[binx, biny])
    z = ret.statistic
    
    # Caclulate the excess or less shots based on league average
    for idx, x in np.ndenumerate(z):
        if x != 0:
            z[idx] = x - int(league_avg_shots_phr)
    
    # Read given image
    img = io.imread('./../figures/nhl_rink.png')
    fig = px.imshow(img)
    fig.add_trace(go.Contour(z=z,
                             x = [0, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100],
                             y = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 430, 467],
                             showscale=True, opacity = 0.5, transpose = True, line_smoothing=1, autocolorscale = False,
                             colorscale = [[0, 'rgb(0,0,255)'], [0.5, 'rgb(255,255,255)'], [1, 'rgb(255,0,0)']],
                             contours=dict(coloring='heatmap', start = -30, end = 30)))
    
    fig.update_layout(title=dict(text=season[0:4] + '-' + season[4:] + ' ' + game_type + ' season - ' + team))
    fig.update_layout(title_pad_l=180)
    fig.update_traces(colorbar_title_text='Excess shots per hour', selector=dict(type='contour'))
    fig['layout']['yaxis1'].update(title='Y axis')
    fig['layout']['xaxis1'].update(title='X axis')
    fig.show()
    #fig.write_html("plotly_demo.html")

# Select based on interactive function
def plotit(season, game_type, team):
    result = []
    if game_type != 'Regular or Playoff':
        if(game_type == 'Regular'):
            result = rdata
        if(game_type == 'Playoff'):
            result = pdata
    
    if season != 'Select a season':
        result = result[result[ 'season' ] == season]
        league_avgerage = league_avg(result)
        
    if team != 'Select a team':
        result = result[result['team name'] == team]
        
    if len(result) > 0:
        fig_preprocess(result, league_avgerage, str(season), game_type, team)
    else:
        print("No data to show for current selection")

In [37]:
# Execute the interactive function
interactive(plotit, season = season, game_type = game_type, team = team)

interactive(children=(Dropdown(description='Season:', options=(20162017, 20172018, 20182019, 20192020, 2020202…